# Learning on Graphs

This tutorial is based on the material of Stanford class [CS224W](https://web.stanford.edu/class/cs224w/index.html)

## Graphs with networkx and Pytorch geometric

As usual, some installations and imports. This may take a while:

In [ ]:
import networkx as nx
import torch
import torch.nn as nn
from torch.optim import SGD
import random
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression


# Install torch geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-2.4.0+cu121.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-2.4.0+cu121.html
!pip install -q torch-geometric

!pip install git+https://github.com/snap-stanford/ogb.git

from torch_geometric.datasets import KarateClub
from torch_geometric.utils import to_networkx

%matplotlib inline
import matplotlib.pyplot as plt

### Load and examine Zachary's Karate Club Network with networkx

NetworkX is one of the most frequently used Python packages to create, manipulate, and mine graphs.

The Karate Club Network is a graph which describes a social network of 34 members of a karate club and documents links between members who interacted outside the club.

In [ ]:
G = nx.karate_club_graph()

# G is an undirected graph
type(G)

In [ ]:
# Visualize the graph
nx.draw(G, with_labels = True)

In [ ]:
# Let's look at a node
nodes = G.nodes(data=True)
nodes[0]

In [ ]:
# How many nodes in the graph? How many edges?
N = G.number_of_nodes()
E = G.number_of_edges()
print(N, " nodes, ", E, " edges")

In [ ]:
# Let's look at an edge
edges = G.edges
ij = (0, 11)
if ij in edges:
  print(edges[ij])
else:
  print("Not in the graph")

In [ ]:
# Degree
node_id = 0
G.degree(node_id)

In [ ]:
# Neighbors
node_id = 0
list(G.neighbors(node_id))

In [ ]:
# Visualization function  for NX graph or PyTorch tensor
def visualize(h, color, epoch=None, loss=None, accuracy=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])

    if torch.is_tensor(h):
        h = h.detach().cpu().numpy()
        plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
    else:
        nx.draw_networkx(h, pos=nx.spring_layout(h, seed=42), with_labels=False,
                         node_color=color, cmap="Set2")
    plt.show()

### Pytorch Geometric

In [ ]:
dataset = KarateClub()
dataset = KarateClub()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

Each graph in PyTorch Geometric is represented by a single [`Data`](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data) object, which holds all the information to describe its graph representation.

In [ ]:
data = dataset[0]  # Get the first graph object.

print(data)

The `edge_index` property holds the information about the **graph connectivity**, *i.e.*, a tuple of source and destination node indices for each edge.
PyG further refers to **node features** as `x` (each of the 34 nodes is assigned a 34-dim feature vector), and to **node labels** as `y` (each node is assigned to exactly one class).

In [ ]:
# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {(data.num_edges) / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.has_isolated_nodes()}')
print(f'Contains self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Adjacency list:

In [ ]:
edges = data.edge_index
print(edges.T)

This representation is known as the **COO format (coordinate format)** commonly used for representing sparse matrices.
Instead of holding the adjacency information in a dense representation $\mathbf{A} \in \{ 0, 1 \}^{|\mathbb{V}| \times |\mathbb{V}|}$, PyG represents graphs sparsely, which refers to only holding the coordinates/values for which entries in $\mathbf{A}$ are non-zero.

In [ ]:
# From geometric to networkx
data_G = to_networkx(data, to_undirected=True)
visualize(data_G, color=data.y)

# Node Embeddings

In this section we will learn some shallow embeddings for the nodes of the Karate graph

In [ ]:
# Seed for reproducibility
seed = 42

def seed_everything(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

seed_everything(seed)

Let's start with randomly initialized 16-dimensional embeddings (one for each node)

In [ ]:
# 34 16-dimensional embeddings:
emb = nn.Embedding(num_embeddings=34, embedding_dim=16)
print(emb.weight.data.shape)

# Initialize with uniform([0,1))
emb.weight.data = torch.rand(size=emb.weight.data.shape)

In [ ]:
# Select an embedding in emb_sample
id = torch.LongTensor([1])
print(emb(id))

In [ ]:
# Select multiple embeddings
ids = torch.LongTensor([1, 3])
print(emb(ids))

We can visualize the embedding in 2D using PCA.

The node attribute "club" tells us which club the member joined after the original club split.

In [ ]:
def visualize_emb(emb):
  X = emb.weight.data.numpy()
  pca = PCA(n_components=2)
  components = pca.fit_transform(X)
  plt.figure(figsize=(6, 6))
  club1_x = []
  club1_y = []
  club2_x = []
  club2_y = []
  for node in G.nodes(data=True):
    if node[1]['club'] == 'Mr. Hi':
      club1_x.append(components[node[0]][0])
      club1_y.append(components[node[0]][1])
    else:
      club2_x.append(components[node[0]][0])
      club2_y.append(components[node[0]][1])
  plt.scatter(club1_x, club1_y, color="red", label="Mr. Hi")
  plt.scatter(club2_x, club2_y, color="blue", label="Officer")
  plt.legend()
  plt.show()

# Visualize the initial random embeddding
visualize_emb(emb)

Let's try to fit a (logistic regression) classifier to predict the club from the node embeddings:

In [ ]:
X = emb.weight.detach().numpy()
y = np.zeros(X.shape[0])
for i in range(G.number_of_nodes()):
  y[i] = G.nodes[i]['club'] == 'Mr. Hi'

clf = LogisticRegression()
clf.fit(X[:27], y[:27])
clf.score(X[27:], y[27:])

We will take a **random walk** view, which will be useful later...

We train the embedding to maximize the likelihood that **similar** nodes are neighbors in a 1-step random walk on the graph, using dot product as our measure of similarity:

$\mathcal{L} = -\sum_{u\in V}\sum_{v\in\mathrm{neighbors(u)}} \log P(v | u) = -\sum_{u\in V}\sum_{v\in\mathrm{neighbors(u)}} \log \left(\frac{\exp(z_u^Tz_v)}{\sum_{n\in V}\exp(z_u^Tz_n)}\right)$

This is approximated by minimizing **binary cross entropy loss** with positive examples (actual neighbors) and negative examples (randomly sampled "non neighbors" $n_i$)

$\mathcal{L} \simeq -\sum_{u\in V}\sum_{v\in\mathrm{neighbors(u)}} \log(\sigma(z_u^Tz_v)) -\sum_{u\in V}\sum_{i=1}^N \log(1-\sigma(z_u^Tz_{n_i}))$

Generate positive examples (the edges of the graph)

In [ ]:
def positive_samples_adj(G):
  edge_list = []
  for edge in G.edges():
    edge_list.append(edge)

  edge_index = torch.tensor(edge_list).T
  return edge_index

pos_edge_index = positive_samples_adj(G)
pos_edge_index.shape

Generate negative examples (n pairs of nodes that do not share an edge in the graph)

In [ ]:
def negative_samples(G, n):
  neg_edge_list = []

  edge_set = set()
  for edge in G.edges():
    edge_set.add(edge)

  nodes = list(G.nodes)
  for i, node1 in enumerate(nodes):
      for node2 in nodes[i+1:]:
          if (node1, node2) not in edge_set:
            neg_edge_list.append((node1, node2))
  neg_edge_list = random.sample(neg_edge_list, n)
  edge_index = torch.tensor(neg_edge_list).T
  return edge_index

neg_edge_index = negative_samples(G, 78)
neg_edge_index.shape

In [ ]:
def accuracy(pred, label):
  return torch.sum(torch.round(pred) == label) / pred.shape[0]

def train(emb, loss_fn, train_label, train_edge,
          epochs=500,
          learning_rate=0.1):
  optimizer = SGD(emb.parameters(), lr=learning_rate, momentum=0.9)

  for i in range(epochs):
    sigmoid = nn.Sigmoid()
    optimizer.zero_grad()
    node_emb = emb(train_edge)
    dot_product = torch.sum(node_emb[0] * node_emb[1], -1)
    result = sigmoid(dot_product)
    loss = loss_fn(result, train_label)
    print("Epoch:", i, "Loss:", loss.item(),
          "Acc:", accuracy(result, train_label).item())
    loss.backward()
    optimizer.step()

  return emb

In [ ]:
loss_fn = nn.BCELoss()

pos_edge_index = positive_samples_adj(G)
neg_edge_index = negative_samples(G, n=pos_edge_index.shape[1])

# Generate the positive and negative labels
pos_label = torch.ones(pos_edge_index.shape[1], )
neg_label = torch.zeros(neg_edge_index.shape[1], )

# Concat positive and negative labels into one tensor
train_label = torch.cat([pos_label, neg_label], dim=0)
n = len(train_label)

# Concat positive and negative edges into one tensor
# Since the network is very small, we do not split the edges into val/test sets
train_edge = torch.cat([pos_edge_index, neg_edge_index], dim=1)

# Shuffle
perm = torch.randperm(n)
train_edge = train_edge[:, perm]
train_label = train_label[perm]

train(emb, loss_fn, train_label, train_edge)

In [ ]:
# Visualize learned embeddings
visualize_emb(emb)

Again, let's try to fit a (logistic regression) classifier to predict the club from the node embeddings:

In [ ]:
X = emb.weight.detach().numpy()
y = np.zeros(X.shape[0])
for i in range(G.number_of_nodes()):
  y[i] = G.nodes[i]['club'] == 'Mr. Hi'

clf = LogisticRegression(random_state=0).fit(X[:27], y[:27])
clf.score(X[27:], y[27:])

**TODO:** RANDOM WALK

Using the negative sampling approach described above, learn a 16-dimensional node embedding based on **first-order unbiased random walks**.

To generate the positive examples, simulate a random walk from each node $u$ in the graph. Start from $u$ and continue the walk by selecting a neighbor of the current node uniformly at random. Every node $v_i$ visited along the walk provides a positive example $(u, v_i)$.

You can generate negative examples at random as before.

In [ ]:
# Implement this function to generate positive examples
# The function takes the graph, the length of walks, and the number of walks to simulate from each node of the graph
def positive_samples_fo(G, length=5, n_walks=10):
  # YOUR CODE HERE
  return ...

In [ ]:
emb_2 = nn.Embedding(num_embeddings=G.number_of_nodes(), embedding_dim=16)
emb_2.weight.data = torch.rand(size=emb_2.weight.data.shape)

pos_edge_index = positive_samples_fo(G, length=5, n_walks=10)
neg_edge_index = negative_samples(G, n=300)

loss_fn = nn.BCELoss()

# Generate the positive and negative labels
pos_label = torch.ones(pos_edge_index.shape[1], )
neg_label = torch.zeros(neg_edge_index.shape[1], )

# Concat positive and negative labels into one tensor
train_label = torch.cat([pos_label, neg_label], dim=0)
n = len(train_label)

# Concat positive and negative edges into one tensor
# Since the network is very small, we do not split the edges into val/test sets
train_edge = torch.cat([pos_edge_index, neg_edge_index], dim=1)

# Shuffle
perm = torch.randperm(n)
train_edge = train_edge[:, perm]
train_label = train_label[perm]

train(emb_2, loss_fn, train_label, train_edge)

visualize_emb(emb_2)

In [ ]:
X = emb_2.weight.detach().numpy()
y = np.zeros(X.shape[0])
for i in range(G.number_of_nodes()):
  y[i] = G.nodes[i]['club'] == 'Mr. Hi'

clf = LogisticRegression(random_state=0).fit(X[:27], y[:27])
clf.score(X[27:], y[27:])

**TODO**: NODE2VEC

Now you can try to implement **second-order biased random walks** like in node2vec.

For each node $v_i$ in the walk, keep track of its predecessor $v_{i-1}$
The neighbors of $v_i$ are assigned non-uniform probabilities (unnormalized):
* The predecessor $v_{i-1}$ is assigned weight $1/p$ (return parameter)
* Nodes that are *not* neighbors of $v_{i-1}$ are assigned weight $1/q$ (walk-away parameter)
* Other neighbors of $v_i$ are assigned weight $1$

Normalized weights give the distribution of the node to visit after $v_i$

In [ ]:
# Implement this function to generate positive examples
# It takes the graph, the length of the walks, the number of walks
# to simulate from each node of the graph, the return parameter p
# and the walk-away parameter q
def positive_samples_so(G, length=5, n_walks=10, p=1, q=2):
  # YOUR CODE HERE
  return ...

In [ ]:
emb_3 = nn.Embedding(num_embeddings=G.number_of_nodes(), embedding_dim=16)

# Initialize with uniform([0,1))
emb_3.weight.data = torch.rand(size=emb_3.weight.data.shape)

pos_edge_index = positive_samples_so(G, length=5, n_walks=10, p=1, q=2)
neg_edge_index = negative_samples(G, 300)

loss_fn = nn.BCELoss()

# Generate the positive and negative labels
pos_label = torch.ones(pos_edge_index.shape[1], )
neg_label = torch.zeros(neg_edge_index.shape[1], )

# Concat positive and negative labels into one tensor
train_label = torch.cat([pos_label, neg_label], dim=0)
n = len(train_label)

# Concat positive and negative edges into one tensor
# Since the network is very small, we do not split the edges into val/test sets
train_edge = torch.cat([pos_edge_index, neg_edge_index], dim=1)

# Shuffle
perm = torch.randperm(n)
train_edge = train_edge[:, perm]
train_label = train_label[perm]

train(emb_3, loss_fn, train_label, train_edge)

In [ ]:
# Visualize learned embeddings
visualize_emb(emb_3)

In [ ]:
X = emb_3.weight.detach().numpy()
y = np.zeros(X.shape[0])
for i in range(G.number_of_nodes()):
  y[i] = G.nodes[i]['club'] == 'Mr. Hi'

clf = LogisticRegression(random_state=0).fit(X[:27], y[:27])
clf.score(X[27:], y[27:])

## Node Classification with Graph Neural Networks

In this section we will implement a Graph Convolutional Neural Network using PyTorch Geometric.

Some (more) imports first:

In [ ]:
import torch
import torch.nn as nn
import random
import numpy as np

from ogb.nodeproppred import PygNodePropPredDataset

from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator


%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Check you are on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

In [ ]:
# Seeding
seed = 42

def seed_everything(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

seed_everything(seed)

### The dataset

The Open Graph Benchmark (OGB) is a collection of realistic, large-scale, and diverse benchmark datasets for machine learning on graphs. Its datasets are automatically downloaded, processed, and split using the OGB Data Loader. The model performance can also be evaluated by using the OGB Evaluator in a unified manner.

 The ogbn-arxiv dataset is a directed graph, representing the citation network between all Computer Science (CS) arXiv papers indexed by MAG. Each node is an arXiv paper and each directed edge indicates that one paper cites another one. Each paper comes with a 128-dimensional feature vector obtained by averaging the embeddings of words in its title and abstract.

The **node classification** task is to predict the 40 subject areas of arXiv CS papers, e.g., cs.AI, cs.LG, and cs.OS, which are manually determined (i.e., labeled) by the paper's authors and arXiv moderators.

In [ ]:
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name=dataset_name,
                                 root='./arxiv',
                                 transform=T.ToSparseTensor())


In [ ]:
# Just one graph
print(len(dataset))

G = dataset[0]

print(G)

print(G.num_features)

print(dataset.num_classes)

It's important to understand our data. The input to the network will be the node features..

In [ ]:
G.x.shape

...but also the graph connectivity:

In [ ]:
G.adj_t.shape

In [ ]:
G.adj_t

### The model

![test](https://drive.google.com/uc?id=128AuYAXNXGg7PIhJJ7e420DoPWKb-RtL)

**TODO:** implement the Graph Convolutional Neural Network outlined above, using torch geometric's implementation of the graph convolution operator as the main building block: [GCNConv](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GCNConv.html)

In [ ]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

# input dim: the number of node features
# hidden dim: number of channels, fixed from the output of the first GCN block to the input of the last one
# output dim: the number of classes
# num_layers: how many times the first block in the picture above is repeated
# dropout: dropout parameter
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout):

        super(GCN, self).__init__()

        # Useful stuff from torch.nn:
        # torch.nn.ModuleList to define lists of modules
        # torch.nn.BatchNorm1d
        # torch.nn.LogSoftmax
        # relu and dropout from torch.nn.functional

    def forward(self, x, adj_t):
        # YOUR CODE HERE
        return ...

The following function implements a training epoch:

In [ ]:
def train(model, data, train_idx, optimizer, loss_fn):
    model.train()
    loss = 0

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)
    loss = loss_fn(out[train_idx], data.y[train_idx].reshape(-1))

    loss.backward()
    optimizer.step()

    return loss.item()

Here is a function to evaluate the model:

In [ ]:
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    out = model(data.x, data.adj_t)

    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

Here are some suggested hyperparmeters:

In [ ]:
args = {
    'num_layers': 3,
    'hidden_dim': 256,
    'dropout': 0.2,
    'lr': 0.05,
    'epochs': 100,
}

We split data for training/validation/testing and prepare our model:

In [ ]:
G = G.to(device)
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

model = GCN(G.num_features, args['hidden_dim'],
            dataset.num_classes, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbn-arxiv')

Time for training:

In [ ]:
import copy

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = F.nll_loss # negative log likelihood

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, G, train_idx, optimizer, loss_fn)
  result = test(model, G, split_idx, evaluator)
  train_acc, valid_acc, test_acc = result
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% ')

Let's see how we did on the test data

In [ ]:
best_result = test(best_model, G, split_idx, evaluator)
train_acc, valid_acc, test_acc = best_result
print(f'Best model: '
      f'Test: {100 * test_acc:.2f}%')